# Data Science z Python 3.10. Metryki i ocena modeli
## 🇬🇧 Data Science with Python 3.10. Metrics and models evaluation
#### 👨‍🏫 PhD Wojciech Oronowicz-Jaśkowiak
#### 🤖 https://github.com/aipogodzinach

### 1. Import bibliotek
#### 🇬🇧 Import libraries
W tym miejscu importujemy biblioteki, które będą nam potrzebne do analizy danych. Jeżeli widoczny będzie błąd, to konieczne jest zainstalowanie stosowanej biblioteki w danym środowisku - wwchodzimy w terminal, aktywujemy środowisko pracy, instalujemy biblioteki (np. pip install pandas). Aby uruchomić ten notatnik należy skorzystać z Google Colab lub ze środowiska lokalnego.

In [27]:
import os
import pandas as pd

from keras.models import Sequential 
from keras.layers import Dense

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import matplotlib as plt
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, f1_score

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [2]:
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/')

Mounted at /content/drive


### 2.Przygotowanie danych
#### 🇬🇧 Data preparation

In [3]:
os.getcwd()

base_data = pd.read_csv("DSP_1.csv")
base_data.columns
cols = ["Survived", "Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]
data = base_data[cols].copy()

data["Age"].fillna((data["Age"].mean()), inplace=True)
data.dropna(subset=['Embarked'], inplace=True)

encoder = LabelEncoder()
data.loc[:,"Sex"] = encoder.fit_transform(data.loc[:,"Sex"])
data.loc[:,"Embarked"] = encoder.fit_transform(data.loc[:,"Embarked"])

y = data.iloc[:,0]
x = data.iloc[:,1:8]
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

### Zadanie

#### 1. Wykorzystaj poniższy kod, żeby wytrenować cztery modele ML korzystając z zaproponowanych architektu sieci neuronowych. Uzupełnij kod dodając do niego zaproponowaną przez siebie architekturę sieci neuronowej.

In [47]:
def model(X_train, y_train):
    
    forest = RandomForestClassifier (n_estimators=20, random_state=0)
    forest.fit(X_train,y_train)
    print("Random Forest: {0}".format(forest.score(X_train,y_train)))
    
    lreg = LogisticRegression(max_iter = 1000)
    lreg.fit(X_train,y_train)
    print("Regresja logistyczna: {0}".format(lreg.score(X_train,y_train)))
  
    tree = DecisionTreeClassifier()
    tree.fit(X_train,y_train)
    print("Decision Tree: {0}".format(tree.score(X_train,y_train)))
    
    neuralnetwork = Sequential([
      Dense(8, activation='relu', input_shape=(7,)),
      Dense(8, activation='relu'),
      Dense(1, activation='sigmoid')
      ])
    neuralnetwork.compile(
          optimizer='adam',
          loss='binary_crossentropy',
          metrics=['accuracy']
    )
    neuralnetwork.fit(X_train, y_train, batch_size=32, epochs = 100, verbose = 0)
    print("Neural Network: {0}".format(neuralnetwork.evaluate(X_train,y_train)[1]))
    return forest, lreg, tree, neuralnetwork

forest, lreg, tree, nn = model(X_train,y_train)

Random Forest: 0.980309423347398
Regresja logistyczna: 0.8213783403656821
Decision Tree: 0.9845288326300985
23/23 [==============================] - 0s 2ms/step - loss: 0.4518 - accuracy: 0.8214
Neural Network: 0.8213783502578735


In [51]:
y1_predict = forest.predict(X_test)
print("Random Forest {0}".format(accuracy_score(y_test, y1_predict)))

y2_predict = lreg.predict(X_test)
print("Logistic Regresion {0}".format(accuracy_score(y_test, y2_predict)))

y3_predict = tree.predict(X_test)
print("Decision Tree {0}".format(accuracy_score(y_test, y3_predict)))

y4_predict = nn.predict(X_test).round()
print("Neural Network {0}".format(accuracy_score(y_test, y4_predict)))

Random Forest 0.7640449438202247
Logistic Regresion 0.7134831460674157
Decision Tree 0.7752808988764045
Neural Network 0.7134831460674157


#### 2. Poszukaj informacji dotyczących możliwości generowania confusion matrix w tensorflow, wygeneruj ją dla każdego z czterech modeli. Który z modeli jest "bardziej skłonny" do popełniania błędów I rodzaju?

In [52]:
print("Random Forest \n{0}".format(confusion_matrix(y_test, y1_predict)))
print("Logistic Regresion \n{0}".format(confusion_matrix(y_test, y2_predict)))
print("Decision Tree \n{0}".format(confusion_matrix(y_test, y3_predict)))
print("Neural Network \n{0}".format(confusion_matrix(y_test, y4_predict)))

Random Forest 
[[90 15]
 [27 46]]
Logistic Regresion 
[[84 21]
 [30 43]]
Decision Tree 
[[90 15]
 [25 48]]
Neural Network 
[[81 24]
 [27 46]]


#### 3. Dokładność klasyfikacji nie jest najlepszą metryką w przypadku porównywania ze sobą różnych modeli. Do tego celu bardziej nadaje się metryka F-1. Podaj wyniki F-1 dla każdego modelu i wskaż, który z nich jest najlepszy. Czy najlepszy wynik F-1 występuje również w przypadku modelu, który cechował się najwyższą dokładnością?

In [53]:
print("Random Forest \n{0}".format(f1_score(y_test, y1_predict)))
print("Logistic Regresion \n{0}".format(f1_score(y_test, y2_predict)))
print("Decision Tree \n{0}".format(f1_score(y_test, y3_predict)))
print("Neural Network \n{0}".format(f1_score(y_test, y4_predict)))

Random Forest 
0.6865671641791045
Logistic Regresion 
0.6277372262773723
Decision Tree 
0.7058823529411765
Neural Network 
0.6433566433566433
